In [ ]:
while True: pass

In [ ]:
!git clone https://github.com/setiawansandi/image_recognition.git

Cloning into 'image_recognition'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.
